# simrank优化算法

In [1]:
import numpy as np
import time

## 数据预处理

In [2]:
def getGraph(edge_num):
    with open('test/web-Stanford.txt') as log_fp:
        logs = []
        for i in range(edge_num):
            log = log_fp.readline()
            logs.append(log.strip())    #strip是去掉空格和换行
    
    #from to元祖
    logs_tuple = [ tuple(log.split('\t')) for log in logs ]
    
    #获取from集合和to集合
    from_ = list(set([ log[0] for log in logs_tuple ]))
    to_ = list(set( [log[1] for log in logs_tuple ]))
    
    #顶点集合
    vertex = list(set(from_ + to_))
    
    #图结构邻接矩阵表示
    #图的所有节点都放进去
    #即A矩阵
    graph = np.matrix(np.zeros([len(vertex), len(vertex)]))

    #给图邻接矩阵添加边
    #有向图，非对称
    for log in logs_tuple:
        i = vertex.index(log[0])
        j = vertex.index(log[1])
        graph[i, j] = 1   #行为出度，列为入度
        
    return graph, vertex

## simrank

In [3]:
graph, vertex = getGraph(200)

In [4]:
print(len(vertex))
graph

173


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
#初始化相似度迭代矩阵
sim_mat = np.matrix(np.identity(len(vertex)))
sim_mat

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]])

In [6]:
#计算出度和
def getOutgoingSum(a):
    return graph[a].sum()

#计算入度和
def getInputSum(a):
    return graph[:, a].sum()

#获取a的入度节点
def getInputNode(a):
    index = []
    mat_col = graph[:, a]
    for j in range(len(vertex)):
        if mat_col[j] != 0:
            index.append(j)
    return index

#获取a的出度节点
def getOutgoingNode(a):
    index = []
    for i in range(len(vertex)):
        if graph[a, i] != 0:
            index.append(i)
    return index

In [7]:
#计算δ函数
def getDerList(C, K, Der):
    der = []
    for i in range(K):
        der.append(Der / (K * pow(C, K-i-1))) #计算公式
    return der

#计算a节点的入度部分和
#就是相似度
def getPartial(a):
    partial = []
    #先获取a节点的入度下标
    index = getInputNode(a)
            
    #计算partial_a(i)        
    for i in range(len(vertex)):
        if len(index) == 0:
            partial.append(0)
        else:
            result = 0
            for x in index:
                result = result + sim_mat[x, i]
            partial.append(result)
    return partial

#计算a的基本节点对
def getEssential(a):
    b_list = set()
    #先获取a节点的入度下标
    index = getInputNode(a)
    
    #获取与入度相似度不为0的节点
    temp = set()
    if len(index) != 0:
        for x in index:
            for i in range(len(vertex)):
                if sim_mat[x, i] != 0:
                    temp.add(i)
    temp = list(temp)
    
    #然后获取temp节点的出度，就是基本节点对
    if len(temp) != 0:
        for x in temp:
            index = getOutgoingNode(x)
            if len(index) != 0:
                for i in index:
                    b_list.add(i)
    b_list = list(b_list)
    
    return b_list

#更新相似度矩阵
def calculate(a, b, der, partial, C):
    if a == b:
        return 1
    if getInputSum(a) == 0 or getInputSum(b) == 0:
        return 0
    #前缀系数
    prefix = C / (getInputSum(a) * getInputSum(b))
    #后缀系数
    postfix = 0
    index = getInputNode(b) #b的入度节点
    if len(index) != 0:
        for i in index:
            postfix += partial[i]
    result = prefix * postfix
    #看看result是否大于阈值
    if result > der or sim_mat[a, b] != 0:
        return result
    
    return 0

In [8]:
#simrank优化算法
#三个参数，C是衰败因子，K是迭代次数，Der是阈值误差
def simrank(C, K, Der):
    global sim_mat
    #通过Der计算δ参数，返回数组
    der = getDerList(C, K, Der)
    
    start = time.process_time()
    #迭代次数
    for k in range(K):
        #中间相似度矩阵
        new_note_sim = np.matrix(np.identity(len(vertex)))
        
        #遍历每一个节点a
        for a in range(len(vertex)):
            #如果该节点出度为0（即矩阵行和为0），只需要在最后一次迭代的时候计算。
            #继续下一个a
            if getOutgoingSum(a) == 0 and k != K-1:
                continue
            #计算a节点的入度部分和
            partial = getPartial(a)
            #计算a节点的基本节点对
            b_list = getEssential(a)
            #更新（a,b）的相似度
            for b in b_list:
                new_note_sim[a, b] = calculate(a, b, der[k], partial, C)
                
        #一次迭代完，更新sim_mat
        sim_mat = new_note_sim
    
    end = time.process_time()
    print("运行时间：", end-start)

In [11]:
simrank(0.8, 5, 0.05) #der取0，相当于不设阈值
sim_mat

运行时间： 38.547847100000006


matrix([[1. , 0.8, 0. , ..., 0.8, 0. , 0. ],
        [0.8, 1. , 0. , ..., 0.8, 0. , 0. ],
        [0. , 0. , 1. , ..., 0. , 0. , 0. ],
        ...,
        [0.8, 0.8, 0. , ..., 1. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 1. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 1. ]])